# Import Python libraries

In [2]:
import xlwings as xw
import pandas as pd
import numpy as np
from poes.model.utils import param_poes

In [3]:
xb = xw.Book('poes/controller/control.xlsm')

In [4]:
sheet = xb.sheets['Resumen']

In [5]:
df_poes = sheet['A2'].options(pd.DataFrame, index=False, expand='table').value
df_poes

,Variables,Data values,Dist.,Loc,Scale,Sc,Lim. Min.,Lim. Max.
0,Area,340.00,None,None,None,None,None,None
1,Espesor,40.00,None,None,None,None,None,None
2,Porosida,0.16,None,None,None,None,None,None
3,Swi,0.51,None,None,None,None,None,None
4,Boi,2.60,None,None,None,None,None,None


# Exploratory data analysis

In [6]:
df_poes.columns

Index(['Variables', 'Data values', 'Dist.', 'Loc', 'Scale', 'Sc', 'Lim. Min. ',
       'Lim. Max.'],
      dtype='object')

In [7]:
df_poes.columns = df_poes.columns.str.strip()

In [8]:
df_poes.columns

Index(['Variables', 'Data values', 'Dist.', 'Loc', 'Scale', 'Sc', 'Lim. Min.',
       'Lim. Max.'],
      dtype='object')

In [9]:
df_poes['Dist.'] = ['Triangular', 'Triangular', 'Log Normal', 'Normal', 'Exponencial']
df_poes

,Variables,Data values,Dist.,Loc,Scale,Sc,Lim. Min.,Lim. Max.
0,Area,340.00,Triangular,None,None,None,None,None
1,Espesor,40.00,Triangular,None,None,None,None,None
2,Porosida,0.16,Log Normal,None,None,None,None,None
3,Swi,0.51,Normal,None,None,None,None,None
4,Boi,2.60,Exponencial,None,None,None,None,None


In [10]:
df_poes.shape

(5, 8)

In [11]:
df_poes['Loc'] = np.array([400, 0, 0, 0.4, 1])
df_poes['Scale'] = np.array([600, 90, 0.2, 0.2, 0.2])
df_poes['Sc'] = np.array([0.3, 0.3, 0.8, 0, 0.7])
df_poes['Lim. Min.'] = np.array([50, 0, 0, 0, 1])
df_poes['Lim. Max.'] = np.array([0, 180, 0.4, 1, 2])
df_poes

,Variables,Data values,Dist.,Loc,Scale,Sc,Lim. Min.,Lim. Max.
0,Area,340.00,Triangular,400.0,600.0,0.3,50,0.0
1,Espesor,40.00,Triangular,0.0,90.0,0.3,0,180.0
2,Porosida,0.16,Log Normal,0.0,0.2,0.8,0,0.4
3,Swi,0.51,Normal,0.4,0.2,0.0,0,1.0
4,Boi,2.60,Exponencial,1.0,0.2,0.7,1,2.0


# Use Loc method from pandas to extract specific values

In [12]:
df_poes.loc[2, "Data values"]

0.16

In [13]:
df_poes.loc[3, "Variables"]

'Swi'

# Call param_poes function

In [14]:
df_poes.columns

Index(['Variables', 'Data values', 'Dist.', 'Loc', 'Scale', 'Sc', 'Lim. Min.',
       'Lim. Max.'],
      dtype='object')

In [15]:
df_poes

,Variables,Data values,Dist.,Loc,Scale,Sc,Lim. Min.,Lim. Max.
0,Area,340.00,Triangular,400.0,600.0,0.3,50,0.0
1,Espesor,40.00,Triangular,0.0,90.0,0.3,0,180.0
2,Porosida,0.16,Log Normal,0.0,0.2,0.8,0,0.4
3,Swi,0.51,Normal,0.4,0.2,0.0,0,1.0
4,Boi,2.60,Exponencial,1.0,0.2,0.7,1,2.0


In [16]:
param_poes(df_poes, 1, 'Dist.', 'Loc', 'Scale', 100, 'Sc', 'Lim. Min.', 'Lim. Max.')

array([24.35770081, 31.85968019, 48.78312637, 32.15714643, 65.20795161,
       48.80448798, 46.41390706, 73.65590076, 41.70499795, 63.49586467,
       66.49895656, 33.06127923, 24.30107234, 34.57862179, 50.14987262,
       44.91946477, 52.04178029, 33.21661588, 10.94996697, 38.23087706,
       42.16735664, 73.03920266, 29.99795467, 22.87080275, 24.01224062,
       15.1135185 , 25.91225885, 25.22504481, 72.48916776, 28.32101971,
       57.00285494, 72.04044338, 64.53476915, 30.35597476, 51.80722083,
       35.33161251, 50.7482378 , 39.29616834, 43.58660874, 28.43094126,
       12.45509605, 68.04297834, 16.835685  , 52.99156655, 61.27757658,
       31.87774501, 30.03502196, 73.37723346, 58.00552957, 13.44567874,
        8.82698399, 58.21286466, 59.44218179, 43.80820071, 38.3300098 ,
       58.90899839, 40.14711408, 42.2903535 , 15.99140476, 65.18085671,
       16.44170614, 25.81896422, 26.31664602, 31.52826876, 64.12738531,
       59.47295424, 32.63132189, 17.07006981, 15.67970658, 31.03